In [1]:
import itertools
import numpy as np
import pandas as pd

In [3]:
IVdata = pd.read_stata("./data/IV.dta")

In [4]:
independent = ["Fintech", "Size", "LEV", "Growth", "CapEx", "PPE", "Indep", "GDP", "Population"]
IV = ["FintechIV", "Size", "LEV", "Growth", "CapEx", "PPE", "Indep", "GDP", "Population"]

In [5]:
year_industry = []
for _ in itertools.product(IVdata.year.unique(), IVdata.industry.unique()):
    year_industry.append(_)

In [6]:
year_industry_dummy = np.zeros((len(IVdata), len(year_industry)))
year_ind_num = []
for i in range(len(IVdata)):
    year_ind = (IVdata.iloc[i].year, IVdata.iloc[i].industry)
    for j in range(len(year_industry)):
        if year_ind == year_industry[j]:
            year_ind_num.append(j)
            year_industry_dummy[i][j] = 1
            break

In [7]:
IVdata["year_ind_num"] = year_ind_num

In [8]:
X = np.hstack([IVdata.groupby("code").apply(lambda x: x[independent]-x[independent].mean()).values, year_industry_dummy])

In [9]:
Z = np.hstack([IVdata.groupby("code").apply(lambda x: x[IV]-x[IV].mean()).values, year_industry_dummy])

In [10]:
Y = IVdata.Patent.values

In [11]:
X = np.matrix(X)
Z = np.matrix(Z)
Y = np.matrix(Y).T

#### Fixed_effect model (baseline)

stata code
```
xtset code year

xtreg Patent Fintech Size LEV Growth CapEx PPE Indep GDP Population i.year#i.industry, fe vce(cluster city)
```

In [12]:
invXX = np.linalg.pinv(X.T.dot(X)) 
XY = X.T * Y
bhat_ols = invXX * XY

In [13]:
uhat = Y - X*bhat_ols
uhats = np.multiply(X,uhat)
bvar_ols = invXX * (uhats.T * uhats) * invXX
bse_ols = np.sqrt(np.diag(bvar_ols))

c:\users\56236\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in sqrt
  after removing the cwd from sys.path.


#### Fix_effect model + IV

stata code

```
xtset code year

xtreg Patent Fintech Size LEV Growth CapEx PPE Indep GDP Population i.year#i.industry, fe vce(cluster city)

```

In [15]:
invZX = np.linalg.pinv(Z.T.dot(X)) 
ZY = Z.T * Y
bhat_ols = invZX * ZY

In [16]:
uhat = Y - X*bhat_ols
uhats = np.multiply(Z,uhat)
bvar_ols = invZX * (uhats.T * uhats) * invZX
bse_ols = np.sqrt(np.diag(bvar_ols))

c:\users\56236\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in sqrt
  after removing the cwd from sys.path.
